In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['LD_LIBRARY_PATH'] = "$LD_LIBRARY_PATH:/usr/local/cuda-11.3/lib64/"
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
#import torch
#import torch.nn as nn
#from torch.utils.data import DataLoader
#import pandas as pd
import networkx as nx
import osmnx as ox
from models.hrnr_original.utils import *
from models.hrnr_original.conf import beijing_hparams
# from models.hrnr_original.train import train_struct_cmt, train_fnc_cmt_rst
from models.hrnr_original.model import *
import torch
import torch.nn.functional as F
from torch import optim
from models.training.hrnr_data_generation import *
from models import HRNRModel

In [2]:
torch.cuda.set_device(1)

In [3]:
# try on original
hparams = dict_to_object(beijing_hparams)
hparams.device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
adj, features, sp_label = load_gae_data(hparams) 

/dstore/home/pheinemeyer/Road-Network-Embedding-Generator/models/training


In [ ]:
print(len(np.unique(features[:, 2])))

In [ ]:
train_struct_cmt()

In [ ]:
train_fnc_cmt_rst()

In [4]:
# load road network
network = RoadNetwork()
network.load("../../osm_data/porto")
data = network.generate_road_segment_pyg_dataset()


In [5]:
trajectory = Trajectory(
    "../../datasets/trajectories/Porto/road_segment_map_final.csv", nrows=10000000
)

Pandas Apply:   0%|          | 0/1544234 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1544234 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1544234 [00:00<?, ?it/s]

In [6]:
adjp, featuresp, sp_labelp, t_adjp = get_data(network, trajectory, remove_highway_label=True)

100%|██████████| 1544234/1544234 [11:16<00:00, 2282.40it/s]


In [7]:
# train structural region and functional region matrices
train_struct_cmt_custom(adjp, featuresp, sp_labelp)
train_fnc_cmt_rst_custom(adjp, featuresp, sp_labelp, t_adjp)

47.425174713134766
30.560537338256836
27.412315368652344
24.913055419921875
23.244224548339844
21.646198272705078
19.57803726196289
20.550161361694336
20.61646270751953
21.151586532592773
16.121976852416992
16.637510299682617
17.944889068603516
16.81360626220703
15.21816349029541
16.76313018798828
19.8785457611084
16.387723922729492
14.066655158996582
14.971779823303223
18.91792106628418
14.465333938598633
14.216695785522461
12.516568183898926
11.632509231567383
14.431963920593262
8.55066967010498
11.874552726745605
9.724834442138672
11.518599510192871
10.55440902709961
5.787093162536621
9.767953872680664
7.537724018096924
8.520992279052734
6.252377510070801
11.54685115814209
7.931877136230469
7.882574081420898
8.416472434997559
8.132292747497559
9.862613677978516
8.24179744720459
8.496569633483887
6.14249324798584
14.380117416381836
11.50078010559082
8.148992538452148
6.5418195724487305
13.046992301940918
6.638294219970703
7.427100658416748
11.605703353881836
5.596280574798584
8.14029

In [14]:
# Build model
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = HRNRModel(data, device=device, network=network, data_path="hrnr_data", remove_highway_label=False)

In [15]:
model.train()

Epoch: 50, avg_loss: 6.111485638618469
Epoch: 100, avg_loss: 4.28552446603775
Epoch: 150, avg_loss: 3.5400292030970255
Epoch: 200, avg_loss: 3.143178688287735
Epoch: 250, avg_loss: 2.908280337333679
Epoch: 300, avg_loss: 2.740831860303879
Epoch: 350, avg_loss: 2.6189101208959307
Epoch: 400, avg_loss: 2.5277511045336722
Epoch: 450, avg_loss: 2.4563299496968587
Epoch: 500, avg_loss: 2.472583631038666
Epoch: 550, avg_loss: 2.420749469236894
Epoch: 600, avg_loss: 2.375962265928586
Epoch: 650, avg_loss: 2.3382037129768958
Epoch: 700, avg_loss: 2.303802023104259
Epoch: 750, avg_loss: 2.274268802642822
Epoch: 800, avg_loss: 2.248410801887512
Epoch: 850, avg_loss: 2.225339696828057
Epoch: 900, avg_loss: 2.207201428678301
Epoch: 950, avg_loss: 2.1890170867819534


In [13]:
model.save_model("../model_states/hrnr")

In [16]:
z = model.load_emb()

In [17]:
print(z.shape)

(11331, 128)
